In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: aml-workspace
Azure region: westeurope
Subscription id: 6eee9ed4-ef02-4ad0-bbbd-194153ad2069
Resource group: aml-resources


# Upload and Register Data

In [6]:
'''

### CREATE A TABULAR DATASET
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./bankmarketing_train.csv'], # Upload the diabetes csv files in /data
                        target_path='bankmarketing_train-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'bankmarketing_train-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='bankmarketing_train dataset',
                                description='bankmarketing_train data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

'''

Uploading an estimated of 1 files
Uploading ./bankmarketing_train.csv
Uploaded ./bankmarketing_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Dataset registered.


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [20]:
import os, shutil

# Create a folder for the experiment files
folder_name = 'training'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('train.py', os.path.join(folder_name, "train.py"))
shutil.copy('conda_dependencies.yml', os.path.join(folder_name, "conda_dependencies.yml"))

'training/conda_dependencies.yml'

In [21]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling({
        "--C": uniform(0.005, 0.01),
        "--max_iter":choice(6, 8, 10) 
     } )

# Specify a Policy
policy = BanditPolicy(slack_amount=0.2,
                      evaluation_interval=1,
                      delay_evaluation=5)

#if "training" not in os.listdir():
#    os.mkdir("./training")
#shutil.move('train.py', './training')
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path=experiment_folder +'/conda_dependencies.yml')

# Get the training dataset
#diabetes_ds = ws.datasets.get("bankmarketing_train dataset")


# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=experiment_folder,
                                script='train.py',
                                compute_target = training_cluster,
                                environment=sklearn_env)

'''
est =SKLearn(source_directory=experiment_folder,
                compute_target=training_cluster,
                entry_script='train.py')
'''
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,#estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=6,
                                     max_concurrent_runs=4
                                     )

In [22]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

# Run the experiment
experiment = Experiment(workspace=ws, name='bankmarketing-hyperdrive')
run = experiment.submit(config=hyperdrive_config)

# Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_31f56b65-1875-48b0-8083-16fa9b8e13a2',
 'target': 'aml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-27T17:06:28.472697Z',
 'endTimeUtc': '2022-01-27T17:21:36.805991Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'ed6030f9-9054-4c34-99b9-330edd771827',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1064-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.37.0',
  'space_size': 'infinite_space_size',
  'score': '0.9035912999494183',
  'best_child_run_id': 'HD_31f56b65-1875-48b0-8083-16fa9b8e13a2_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://amlworkspace4469939454.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_31f56b65-1875-48b0-

In [23]:
# Print all child runs, sorted by the primary metric
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# Get the best run, and its metrics and arguments
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
#print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

{'run_id': 'HD_31f56b65-1875-48b0-8083-16fa9b8e13a2_5', 'hyperparameters': '{"--C": 0.009412359545906728, "--max_iter": 10}', 'best_primary_metric': 0.9035912999494183, 'status': 'Completed'}
{'run_id': 'HD_31f56b65-1875-48b0-8083-16fa9b8e13a2_0', 'hyperparameters': '{"--C": 0.009899725403121862, "--max_iter": 10}', 'best_primary_metric': 0.9035912999494183, 'status': 'Completed'}
{'run_id': 'HD_31f56b65-1875-48b0-8083-16fa9b8e13a2_2', 'hyperparameters': '{"--C": 0.0054216570233347, "--max_iter": 10}', 'best_primary_metric': 0.9035912999494183, 'status': 'Completed'}
{'run_id': 'HD_31f56b65-1875-48b0-8083-16fa9b8e13a2_3', 'hyperparameters': '{"--C": 0.005171979286351447, "--max_iter": 8}', 'best_primary_metric': 0.8904400606980273, 'status': 'Completed'}
{'run_id': 'HD_31f56b65-1875-48b0-8083-16fa9b8e13a2_1', 'hyperparameters': '{"--C": 0.005581774461084699, "--max_iter": 8}', 'best_primary_metric': 0.8904400606980273, 'status': 'Completed'}
{'run_id': 'HD_31f56b65-1875-48b0-8083-16fa9

In [24]:
import joblib
# Get your best run and save the model from that run.

#get the best run
best_run = run.get_best_run_by_primary_metric()

best_hyperdrive_model = best_run.register_model(
    model_name="bankmarketing_model",
    model_path="outputs/bankmarketing_model.pkl",
    tags=best_run.get_metrics(),
    properties={'Accuracy': best_run_metrics['Accuracy']}
    )


In [25]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds =TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
#ds = Dataset.Tabular.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [26]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [36]:
ds_frame=ds.to_pandas_dataframe()

In [28]:
y.name

'y'

In [37]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=ds_frame,
    label_column_name="y",
    n_cross_validations=5)

In [38]:
# Submit your automl run

experiment = Experiment(ws, "automl_bankmarketing_experiment")
automl_run = experiment.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_bankmarketing_experiment,AutoML_9f293b83-7ae5-4bd4-a518-b41a5f4a56dd,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl_bankmarketing_experiment,AutoML_9f293b83-7ae5-4bd4-a518-b41a5f4a56dd,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |yes                             |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_9f293b83-7ae5-4bd4-a518-b41a5f4a56dd',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-01-27T18:10:06.473304Z',
 'endTimeUtc': '2022-01-27T18:42:19.040745Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_bankmarketing_experiment","subscription_id":"6eee9ed4-ef02-4ad0-bbbd-194153ad2069","resource_group":"aml-resources","workspace_name":"aml-workspace","region":"westeurope","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enabl

In [39]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: automl_bankmarketing_experiment,
Id: AutoML_9f293b83-7ae5-4bd4-a518-b41a5f4a56dd_26,
Type: None,
Status: Completed)

Best Model Definition:
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('logisticregression', LogisticRegression(C=51.79474679231202, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('sgdclassifierwrapper', SGDClassifierWrapper(alpha=7.551044897959

In [40]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='automl_bankmarketing_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

automl_bankmarketing_model version: 1
	 Training context : Auto ML
	 AUC : 0.9478827682847509
	 Accuracy : 0.9182094081942337


bankmarketing_model version: 1
	 Regularization Strength: : 0.009899725403121862
	 Max iterations: : 10
	 Accuracy : 0.9035912999494183
	 Accuracy : 0.9035912999494183


